In [1]:
!pip install transformers
!pip install torch
!pip install sentencepiece

     |████████████████████████████████| 2.6 MB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.9 MB/s 
     |████████████████████████████████| 636 kB 70.3 MB/s 
     |████████████████████████████████| 895 kB 67.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 7.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/TFG/AskRoberta

/content/drive/MyDrive/Colab Notebooks/TFG/AskRoberta


In [4]:
""" Script for asking Roberta about the domain of a text or definition

Usage:                                                                                                                                     
    cat input-wordnet.txt | python askRoberta4domain.py babel.txt
    cat input-wordnet.txt | python askRoberta4domain.py domains.txt

"""

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import sys
from pprint import pprint

if len(sys.argv) < 2:
    print('Usage:\tpython askRoberta4domain.py topics.txt input_file.txt\n\tpython askRoberta4domain.py topics.txt < input_file.txt')
    exit(1)

# model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli')
# tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')

# model = AutoModelForSequenceClassification.from_pretrained('bart-large-mnli')
# tokenizer = AutoTokenizer.from_pretrained('bart-large-mnli')

model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli')
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

model.eval()

def get_topic(context, topics, tokenizer=tokenizer, model=model):
    with torch.no_grad():
        sentences = [f'{context} {tokenizer.sep_token} The domain of the definition is about {topic}.' for topic in topics]
        input_ids = tokenizer.batch_encode_plus(sentences, pad_to_max_length=True)
        output_probs = torch.softmax(model(torch.tensor(input_ids['input_ids']))[0][:,-1], 0).numpy()
        output = sorted(list(zip(output_probs, topics)), reverse=True)
    return output

with open("frequent-domains.txt", 'rt') as f:
    topics = [topic.rstrip() for topic in f]

input_stream = open("50defs_con_dominio.txt", 'rt') if len(sys.argv) == 3 else sys.stdin

f = open("50askRoberta-result.txt", "w")

for line in input_stream:
    line = line.rstrip()
    topic_dist = get_topic(line, topics)
    print(line)
    f.write(line + '\n')
    pprint(topic_dist)
    pprint(topic_dist, stream=f)
    print()
    f.write('\n')

f.close()

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


all-Pro: Designating, relating to, or consisting of the professional players considered to be the best in their positions, usually as determined by a vote of sportswriters, especially designating a (notional) team made of up such players.
[(0.78574854, 'Sport'),
 (0.012423487, 'American_Football'),
 (0.0069727576, 'Statistics'),
 (0.0069457255, 'Heraldry'),
 (0.0067862906, 'Military'),
 (0.0065917964, 'Genetics'),
 (0.0064265183, 'Logic'),
 (0.0063451966, 'Cricket'),
 (0.006266965, 'Printing'),
 (0.005681799, 'Anatomy'),
 (0.005552107, 'Greek_Mythology'),
 (0.0054421653, 'Entomology'),
 (0.0054240664, 'Psychology'),
 (0.0052173086, 'Anthropology'),
 (0.005078719, 'Phonetics'),
 (0.0048590936, 'Grammar'),
 (0.0048277993, 'Physics'),
 (0.0044893543, 'Mineralogy'),
 (0.0044804215, 'Biology'),
 (0.0043254904, 'Baseball'),
 (0.004325379, 'Linguistics'),
 (0.004309798, 'Economics'),
 (0.0042777397, 'Physiology'),
 (0.004246774, 'Photography'),
 (0.004143428, 'Philosophy'),
 (0.0041276678, 'P

# Compara los resultados de los dominios de Oxford y la primera clasificación de ask Roberta. Devuelve OK si los dominios coinciden, FAIL si no.

In [5]:
def compare_domains(faskrob, foxford, fresult):
  try:
    words = 0
    good = 0
    for line in foxford:
      line1 = line.strip() # delete the newline at the end
      for part1 in line1.split():
        if ":" in part1:
          words+=1
          end = len(line1)
          linea1 = list(line1[0:end])
          for idx1 in range(len(linea1)):
            if linea1[idx1] is ':':
              dom1 = line1[idx1+2:end] # dominio oxford

              # mirar si hay mas de un dominio en esa linea
              ldom = list(dom1)
              numdom = 1
              for i in range(len(ldom)):
                if ldom[i] is ' ':
                  dom11 = "".join(ldom[0:i])
                  dom12 = "".join(ldom[i+1:end])
                  numdom = 2

              break
          b = False
          for line2 in faskrob:
            if b is False:
              for part2 in line2.split():
                if "[" in part2:
                  end = len(line2)
                  linea2 = list(line2[0:end])
                  for idx2 in range(len(linea2)):
                    if linea2[idx2] is ",":
                      num = line2[2:idx2]
                      dom2 = line2[idx2+3:end-4] # dominio roberta
                      break
                  if (numdom is 1 and dom1 == dom2) or (numdom is 2 and (dom11 == dom2 or dom12 == dom2)):
                    print(line1 + ', {} '.format(num) + dom2 + '  OK')
                    fresult.write(line1 + ', {} '.format(num) + dom2 + '  OK\n')
                    good+=1
                  else:
                    print(line1 + ', {} '.format(num) + dom2 + '  FAIL')
                    fresult.write(line1 + ', {} '.format(num) + dom2 + '  FAIL\n')

                  if words % 10 is 0:
                    print('\n')
                    fresult.write('\n')
                  b = True
            else:
              break
        break
    
    print('\n')
    fresult.write('\n')

    goodper = (100*good) / words
    wrongper = 100 - goodper
    print('{} % of domains are the same.'.format(goodper)) 
    print('{} % of domains are different.'.format(wrongper))
    fresult.write('{} % of domains are the same.\n'.format(goodper))
    fresult.write('{} % of domains are different.'.format(wrongper))   
                
  except:
    print('Something failed in the function compare_domains reading from any of the files')
    fresult.write('Something failed in the function compare_domains reading from any of the files\n\n')
    return {}

In [6]:
faskrob = open("/content/drive/MyDrive/Colab Notebooks/TFG/AskRoberta/50askRoberta-result.txt", "r") #"r" si queremos leer
foxford = open("/content/drive/MyDrive/Colab Notebooks/TFG/AskRoberta/50oxford-domains.txt", "r") #"r" si queremos leer
fresult = open("50domains_compared.txt", "w") #"w" si queremos sobreescribir, "a" si queremos escribir a continuacion del contenido que ya tenga
compare_domains(faskrob, foxford, fresult)
faskrob.close()
foxford.close()
fresult.close()

all-Pro: American_Football, 0.78574854 Sport  FAIL
barrel_vault: Architecture, 0.07252116 Nautical  FAIL
ferial: Christian_Church, 0.050094794 Entomology  FAIL
halophile: Ecology, 0.09192484 Ecology  OK
junk_DNA: Genetics, 0.078209646 Genetics  OK
King_of_Arms: Heraldry, 0.8161704 Heraldry  OK
lentic: Ecology, 0.1453876 Nautical  FAIL
objectual: Logic, 0.049475808 Christian_Church  FAIL
quartz_glass: Mineralogy, 0.13222726 Chemistry  FAIL
variate: Statistics, 0.083225474 Statistics  OK


deliberative_democracy: Politics, 0.117285356 Politics  OK
extra-nidal: Entomology Ecology, 0.073082626 Nautical  FAIL
corridor_of_uncertainty: Cricket, 0.050375767 Cricket  OK
Ginnie_Mae: Finance, 0.15575604 Finance  OK
intertrigo: Medicine, 0.1179939 Physiology  FAIL
main_guard: Military, 0.6584228 Military  OK
nonfeasance: Law, 0.9266529 Law  OK
Oamaru: Geology Building, 0.47309574 Mineralogy  FAIL
pair_production: Physics, 0.09917594 Physics  OK
Wallacea: Zoology, 0.85580504 Zoology  OK


telenceph